In [8]:
def get_hours(iso,peak_type,period):
    start_date,end_date=get_start_end_dates(period)
    hours=calc_hours(iso,peak_type,start_date,end_date)
    adj_hours=daylight_adjust_hours(iso,start_date,end_date,peak_type)
    return [iso,peak_type,start_date.strftime("%Y-%m-%d"),end_date.strftime("%Y-%m-%d"),hours+adj_hours]
        
def get_start_end_dates(period):
    from datetime import datetime,timedelta
    from dateutil.relativedelta import relativedelta
    import re
    daily_pattern=r"\d{4}-\d{1,2}-\d{1,2}" #2018-02-02
    monthly_pattern=r"\d{4}[A-Za-z]{3}" #2018Mar
    quarterly_pattern=r"\d{4}Q[1-4]" #2018Q1
    annually_pattern=r"\d{4}A" #2018A
    
    if re.match(daily_pattern,period):
        start_date=datetime.strptime(period,'%Y-%m-%d')
        end_date=start_date
    elif re.match(monthly_pattern,period):
        start_date=datetime(datetime.strptime(period,"%Y%b").year, datetime.strptime(period,"%Y%b").month, 1)
        end_date=start_date+relativedelta(months=1)-timedelta(days=1)
    elif re.match(quarterly_pattern,period):
        start_date=datetime(int(period[:4]),(int(period[-1])-1)*3+1,1)
        end_date=start_date+relativedelta(months=3)-timedelta(days=1)
    elif re.match(annually_pattern,period):
        start_date=datetime(int(period[:4]),1,1)
        end_date=start_date+relativedelta(years=1)-timedelta(days=1)
    else:
        raise Exception("Unknown period type")
    
    return start_date,end_date

def calc_hours(iso,peak_type,start_date,end_date):
    iso_type=get_iso_type(iso)
    
    if iso_type=="Eastern":
        week_day=[0,1,2,3,4]
    elif iso_type=="Western":
        week_day=[0,1,2,3,4,5]
    nerc_holidays=get_nerc_holidays(start_date.year)
    
    all_dates=get_date_range(start_date,end_date)
    peak_dates=[date for date in all_dates if date.weekday() in week_day and date not in nerc_holidays]
    holi_dates=[date for date in all_dates if date.weekday() not in week_day or date in nerc_holidays]
    
    if peak_type=="onpeak": #non-NERC holiday weekday from HE7 to HE22
        return len(peak_dates)*16
    elif peak_type=="offpeak": #the rest are offpeak
        return len(all_dates)*24-len(peak_dates)*16
    elif peak_type=="flat": #every hour
        return len(all_dates)*24
    elif peak_type=="2x16H": #HE7 to HE22 for the weekend and the NERC holiday
        return len(holi_dates)*16
    elif peak_type=="7x8": #non HE7 to HE22 through the week
        return len(all_dates)*8
    else:
        raise Exception("Unknown peak type")
        
def get_iso_type(iso):
    if iso in ('PJMISO','MISO','ERCOT','SPPISO','NYISO'):
        return 'Eastern'
    elif iso in ('WECC','CAISO'):
        return 'Western'
    else:
        raise Exception("Unknown iso")

def get_nerc_holidays(year):
    import holidays
    from datetime import datetime,timedelta,time
    NERC=["New Year's Day",'Memorial Day','Independence Day','Labor Day','Thanksgiving','Christmas Day']
    nerc_holidays=[]
    for ptr in holidays.US(years=year).items():
        if ptr[1] in NERC:
            if ptr[0].weekday()==6:
                nerc_holidays.append(ptr[0]+timedelta(days=1))
            else:
                nerc_holidays.append(ptr[0])
    nerc_holidays = [datetime.combine(date, time()) for date in nerc_holidays]
    return nerc_holidays

def get_date_range(start_date,end_date):
    from datetime import timedelta
    date_list=[start_date+timedelta(days=x) for x in range((end_date-start_date).days+1)]
    return date_list
    
def get_daylight_saving_dates(year):
    #start at second Sunday in March
    start_date=get_n_sunday_in_month(year,3,2)
    #ends at first Sunday in November
    end_date=get_n_sunday_in_month(year,11,1)
    return start_date,end_date

def get_n_sunday_in_month(year,month,n):
    from datetime import datetime,timedelta,time
    date=datetime(year,month,1)
    while date.weekday()!=6:
        date+=timedelta(days=1)
    date=date+timedelta(days=7*(n-1))
    return date

def daylight_adjust_hours(iso,start_date,end_date,peak_type):
    adj_hours=0
    if iso=="MISO" or peak_type not in ["offpeak","flat","7x8"]:
        return adj_hours
    dls_start,dls_end=get_daylight_saving_dates(start_date.year)
    if start_date<=dls_start<=end_date:
        adj_hours-=1
    if start_date<=dls_end<=end_date:
        adj_hours+=1
    return adj_hours

In [7]:
get_hours("ERCOT", "onpeak", "2019May")

['ERCOT', 'onpeak', '2019-05-01', '2019-05-31', 352]